# PCA on Covariates Simulations Notebook

In [82]:
# Import objects from the setup file
import os
import sys
sys.path.append(os.path.expanduser('~/repo/ECMA-31330-Project/Source'))
from ME_Setup import *

# Packages
import numpy as np
import pandas as pd
import seaborn as sns
from pca import pca
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import FactorAnalysis
import math
from tqdm import tqdm
import regex as re

In [83]:
# Supressing Output
from contextlib import contextmanager
import sys, os

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

In [84]:
# Data frame to store output
output = pd.DataFrame()

# 3,000 observations
N = 3000

# Loop over combinations of: betas, covariances between variables, p numbers of parameters
for beta1 in [0.1,1,10]:
    for beta2 in [0.1,1,10]:
        for covariance in [-0.9,-0.5,0,0.5,0.9]:
            for p in [5,20,50]:
                # Select only the scenarios we actually want to run by default - betas of 1, covariances of 0.5, p of 5... require three of these conditions to be satisfied to run.
                counter = 0
                if beta1 == 1:
                    counter += 1
                if beta2 == 1:
                    counter += 1
                if covariance == 0.5:
                    counter += 1
                if p == 5:
                    counter+=1
                if counter >= 3:
                    # Run with and without transformations
                    for exp_of_var in ['yes','no']:
                        # 1000 simulations
                        for k in tqdm(range(1000)):
                            # Initialize Lists to store coef values for all five methods and the true coef
                            pca_coef = []
                            mismeasured_coef = []
                            mismeasured_allvar_coef = []
                            mismeasured_avg_coef = []
                            iv_coef = []
                            true_val_coef =[]

                            # Create variables
                            vars_mean = [0,0,0]
                            vars_cov = np.array([[1,covariance,0],
                                                 [covariance,1,0],
                                                 [0,0,1]])
                            # Producing 3 variables: x for the variable of interest, the true Z covariate, the random error
                            vars_ = pd.DataFrame(np.random.multivariate_normal(vars_mean, vars_cov, N), columns = ['x','true_z','u'])
                            vars_['y'] = beta1 * vars_['x'] + beta2 * vars_['true_z'] + vars_['u']

                            # Create measurement errors for each of the p measurements of the covariates- mean zero and variance one
                            errors_mean = np.zeros(p)
                            errors_cov = np.zeros((p,p))
                            for i in range(p):
                                for j in range(p):
                                    if i == j:
                                        errors_cov[i,j] = 1

                            errors = np.random.multivariate_normal(errors_mean, errors_cov, N)
                            # Column labels for Z variables (covariates variables mismeasured)
                            z_vars = []
                            for i in range(p):
                                z_vars.append('z'+str(i+1))
                            # Add errors to the true_z to get mismeasured values
                            mismeasured_z = pd.DataFrame(errors, columns = z_vars)
                            for i in mismeasured_z.columns:
                                mismeasured_z[i] = mismeasured_z[i] + vars_['true_z']

                            # Take e to the power of the values for half of the measurements if log_of_var is true
                            if exp_of_var == 'yes':
                                mismeasured_z.iloc[:,int(len(mismeasured_z.columns)/2):] =np.exp(mismeasured_z.iloc[:,int(len(mismeasured_z.columns)/2):])
                                
                            # Do feature scaling (normalize to mean 0 and variance 1) for the mismeasured z
                            # Note that x and y are already normalized by construction
                            scaled_mismeasured_z = mismeasured_z.copy()
                            for i in mismeasured_z.columns:
                                scaled_mismeasured_z[i] = (mismeasured_z[i] - mismeasured_z[i].mean()) / mismeasured_z[i].std()

                            # Suppress output
                            with suppress_stdout():
                                # Use PCA on the mismeasured values
                                pca_model = pca()
                                pca_results = pca_model.fit_transform(scaled_mismeasured_z)
                                pca_z = pca_results['PC']['PC1']

                            # NOTE: in non-pca cases, no need to rescale or normalize since mismeasured variables and x and y have mean 0 and sd 1

                            # Average mismeasured variables:
                            vars_['avg_mismeasured_z'] = mismeasured_z[z_vars].mean(axis=1)

                            # Add relevant variables to vars_ dataframe
                            vars_[mismeasured_z.columns] = mismeasured_z
                            vars_['pca_z'] = pca_z

                            # Single mismeasured covariate results
                            model_mismeasured = sm.OLS(vars_['y'],vars_[['x','z1']])
                            results_mismeasured = model_mismeasured.fit()
                            mismeasured_coef.append(results_mismeasured.params[0])

                            # All Variables Mismeasured Results
                            # Create full list of items to include in regression
                            tot_vars = ['x']
                            tot_vars.extend(z_vars)
                            model_mismeasured_allvar = sm.OLS(vars_['y'],vars_[tot_vars])
                            results_mismeasured_allvar = model_mismeasured_allvar.fit()
                            mismeasured_allvar_coef.append(results_mismeasured_allvar.params[0])

                            # Average Mismeasured Variables Results
                            model_mismeasured_avg = sm.OLS(vars_['y'],vars_[['x','avg_mismeasured_z']])
                            results_mismeasured_avg = model_mismeasured_avg.fit()
                            mismeasured_avg_coef.append(results_mismeasured_avg.params[0])

                            # PCA Results
                            model_pca = sm.OLS(vars_['y'],vars_[['x','pca_z']])
                            results_pca = model_pca.fit()
                            pca_coef.append(results_pca.params[0])

                            # Instrumental Variables Results
                            # Instrument z1 on the other items in the mismeasured df
                            z_string = mismeasured_z.columns[1]
                            for z in mismeasured_z.columns[2:]:
                                z_string = z_string + ' + ' + z
                            # Create the predicted value of z1
                            vars_['pred_z1'] = smf.ols("z1 ~ " + z_string, data = mismeasured_z).fit().predict()
                            iv_results = smf.ols("y ~ x + pred_z1 -1", data = vars_).fit()
                            iv_coef.append(iv_results.params[0])

                            # True Results
                            model_true = sm.OLS(vars_['y'],vars_[['x','true_z']])
                            results_true = model_true.fit()
                            true_val_coef.append(results_true.params[0])

                            # Output Findings
                            new_output = pd.DataFrame()
                            new_output['mismeasured_coef'] = mismeasured_coef
                            new_output['mismeasured_allvar_coef'] = mismeasured_allvar_coef
                            new_output['mismeasured_avg_coef'] = mismeasured_avg_coef
                            new_output['pca_coef'] = pca_coef
                            new_output['true_val_coef'] = true_val_coef
                            new_output['iv_coef'] = iv_coef
                            new_output['covariance'] = vars_cov[0][1]
                            new_output['beta1'] = beta1
                            new_output['beta2'] = beta2
                            new_output['p'] = p
                            new_output['exp_of_var'] = exp_of_var
                            output = output.append(new_output)

output

100%|██████████| 2/2 [00:00<00:00,  2.29it/s]


,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,covariance,beta1,beta2,p,exp_of_var
0,0.399043,0.286126,0.475065,0.329801,0.081933,0.343144,0.5,0.1,1.0,5,yes
0,0.375566,0.277360,0.456796,0.301234,0.086412,0.328577,0.5,0.1,1.0,5,yes
0,0.366631,0.193763,0.194784,0.194833,0.086649,0.218756,0.5,0.1,1.0,5,no
0,0.394229,0.188301,0.186376,0.187594,0.087697,0.200708,0.5,0.1,1.0,5,no
0,1.022695,1.018035,1.036367,1.017710,0.984758,1.020303,0.5,1.0,0.1,5,yes
...,...,...,...,...,...,...,...,...,...,...,...
0,3.756241,2.126662,2.132512,2.134524,1.029810,2.361156,0.5,1.0,10.0,5,no
0,10.318776,10.219309,10.428650,10.246406,10.029898,10.281943,0.5,10.0,1.0,5,yes
0,10.304393,10.207344,10.392587,10.234261,10.040918,10.267341,0.5,10.0,1.0,5,yes
0,10.317509,10.165775,10.165579,10.166668,10.062403,10.192212,0.5,10.0,1.0,5,no


In [85]:
output

,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,covariance,beta1,beta2,p,exp_of_var
0,0.399043,0.286126,0.475065,0.329801,0.081933,0.343144,0.5,0.1,1.0,5,yes
0,0.375566,0.277360,0.456796,0.301234,0.086412,0.328577,0.5,0.1,1.0,5,yes
0,0.366631,0.193763,0.194784,0.194833,0.086649,0.218756,0.5,0.1,1.0,5,no
0,0.394229,0.188301,0.186376,0.187594,0.087697,0.200708,0.5,0.1,1.0,5,no
0,1.022695,1.018035,1.036367,1.017710,0.984758,1.020303,0.5,1.0,0.1,5,yes
...,...,...,...,...,...,...,...,...,...,...,...
0,3.756241,2.126662,2.132512,2.134524,1.029810,2.361156,0.5,1.0,10.0,5,no
0,10.318776,10.219309,10.428650,10.246406,10.029898,10.281943,0.5,10.0,1.0,5,yes
0,10.304393,10.207344,10.392587,10.234261,10.040918,10.267341,0.5,10.0,1.0,5,yes
0,10.317509,10.165775,10.165579,10.166668,10.062403,10.192212,0.5,10.0,1.0,5,no


In [86]:
# Save raw coefficient results
output.to_csv(output_dir + '/n_' + str(N) + '_results.csv')

In [87]:
pd.set_option('display.max_rows', 10)

In [88]:
output['pca_ape'] = (output['pca_coef'] - output['beta1']).abs()/output['beta1']
output['mismeasured_coef_ape'] = (output['mismeasured_coef'] - output['beta1']).abs()/output['beta1']
output['mismeasured_allvar_coef_ape'] = (output['mismeasured_allvar_coef'] - output['beta1']).abs()/output['beta1']
output['mismeasured_avg_coef_ape'] = (output['mismeasured_avg_coef'] - output['beta1']).abs()/output['beta1']
output['iv_coef_ape'] = (output['iv_coef'] - output['beta1']).abs()/output['beta1']

In [89]:
output.groupby(['covariance','beta1','beta2','p','exp_of_var']).mean().reset_index()

,covariance,beta1,beta2,p,exp_of_var,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,-0.9,1.0,1.0,5,no,0.222996,0.502274,0.502976,0.502710,0.988012,0.456290,0.497290,0.777004,0.497726,0.497024,0.543710
1,-0.9,1.0,1.0,5,yes,0.240953,0.381684,0.154333,0.317234,0.986656,0.299647,0.682766,0.759047,0.618316,0.845667,0.700353
2,-0.5,1.0,1.0,5,no,0.704848,0.888955,0.889908,0.889899,0.994405,0.868487,0.110101,0.295152,0.111045,0.110092,0.131513
3,-0.5,1.0,1.0,5,yes,0.688289,0.787294,0.623916,0.773750,0.969266,0.745506,0.226250,0.311711,0.212706,0.376084,0.254494
4,0.0,1.0,1.0,5,no,1.026584,1.017426,1.016822,1.016931,1.015957,1.014559,0.016931,0.026584,0.017426,0.016822,0.014559
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,0.5,1.0,10.0,5,yes,4.042376,3.044577,4.937272,3.261862,0.958148,3.559375,2.261862,3.042376,2.044577,3.937272,2.559375
18,0.5,10.0,1.0,5,no,10.300547,10.132153,10.132051,10.133068,10.029099,10.156297,0.013307,0.030055,0.013215,0.013205,0.015630
19,0.5,10.0,1.0,5,yes,10.311584,10.213327,10.410618,10.240334,10.035408,10.274642,0.024033,0.031158,0.021333,0.041062,0.027464
20,0.9,1.0,1.0,5,no,1.722288,1.433541,1.432780,1.434020,0.983394,1.498038,0.434020,0.722288,0.433541,0.432780,0.498038


In [90]:
output.groupby(['covariance','beta1','beta2','p','exp_of_var']).std().reset_index()

,covariance,beta1,beta2,p,exp_of_var,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,-0.9,1.0,1.0,5,no,0.016439,0.041968,0.039979,0.039979,0.047932,0.060905,0.039979,0.016439,0.041968,0.039979,0.060905
1,-0.9,1.0,1.0,5,yes,0.013570,0.017160,0.035227,0.041469,0.039633,0.026073,0.041469,0.013570,0.017160,0.035227,0.026073
2,-0.5,1.0,1.0,5,no,0.021707,0.003510,0.003885,0.004036,0.012563,0.003923,0.004036,0.021707,0.003510,0.003885,0.003923
3,-0.5,1.0,1.0,5,yes,0.010554,0.016204,0.021724,0.024413,0.004051,0.027991,0.024413,0.010554,0.016204,0.021724,0.027991
4,0.0,1.0,1.0,5,no,0.000094,0.003012,0.004599,0.004694,0.004233,0.012182,0.004694,0.000094,0.003012,0.004599,0.012182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,0.5,1.0,10.0,5,yes,0.117534,0.165531,0.237109,0.356652,0.044486,0.271046,0.356652,0.117534,0.165531,0.237109,0.271046
18,0.5,10.0,1.0,5,no,0.023988,0.047549,0.047415,0.047517,0.047099,0.050792,0.004752,0.002399,0.004755,0.004742,0.005079
19,0.5,10.0,1.0,5,yes,0.010170,0.008460,0.025501,0.008588,0.007792,0.010326,0.000859,0.001017,0.000846,0.002550,0.001033
20,0.9,1.0,1.0,5,no,0.066744,0.048352,0.048160,0.049338,0.005570,0.035276,0.049338,0.066744,0.048352,0.048160,0.035276


Code to produce LaTeX tables:

In [91]:
# Helper functions for formatting latex output
def std_parenth_padder(type_col, col_to_pad):
    if type_col == 'std':
        return('(' + col_to_pad + ')')
    else:
        return(col_to_pad)

def format_level_0(type_col, col_to_format):
    if type_col == 'std':
        return('')
    else:
        return(col_to_format)

In [92]:
# Pass this function a pandas dataframe of which to create the LaTeX file
# dataframe is the df
# filename is the save location
# index tells if you want the index saved to the file or not
def Minimal_Latex(dataframe, filename, index = False):

    # Open file
    with open(filename, "w") as f:
        
        # Start from the pandas
        corrected_table = dataframe.to_latex(header = False, index = index)

        # Strip all material that pandas inserts- the tabular environment and toprule and bottomrule
        corrected_table = re.sub(r'\\begin.+', '', corrected_table)
        corrected_table = re.sub(r'\\toprule', '', corrected_table)
        corrected_table = re.sub(r'\\bottomrule', '', corrected_table)
        corrected_table = re.sub(r'\\end.+', '', corrected_table)

        # Write to file and skip blank lines
        f.write(corrected_table.strip())

In [93]:
coef_names_mapper = {'mismeasured_coef':'Single Measurement', 'mismeasured_allvar_coef':'All Measurements', 'pca_coef':'PCA', 'mismeasured_avg_coef':'Average of Measurements', 'iv_coef':'Instrumental Variable'}
ape_names_mapper = {'mismeasured_coef_ape':'Single Measurement', 'mismeasured_allvar_coef_ape':'All Measurements', 'pca_ape':'PCA', 'mismeasured_avg_coef_ape':'Average of Measurements', 'iv_coef_ape':'Instrumental Variable'}

In [94]:
all_params = ['beta1', 'beta2', 'covariance', 'p']
pd.set_option('display.max_rows', 25)

def make_param_tables(dataframe, parameter, exp):

    if parameter != 'beta1':
        dataframe = dataframe.loc[(dataframe['beta1']==1)]
    if parameter != 'beta2':
        dataframe = dataframe.loc[(dataframe['beta2']==1)]
    if parameter != 'covariance':
        dataframe = dataframe.loc[(dataframe['covariance']==0.5)]
    if parameter != 'p':
        dataframe = dataframe.loc[(dataframe['p']==5)]
    dataframe = dataframe.loc[(dataframe['exp_of_var']==exp)]

    exp_mapper = {'yes':'exp', 'no':'no_exp'}

    dataframe = (dataframe.drop(columns = [variable for variable in all_params if variable != parameter])
                          .groupby(parameter)
                          .agg(['mean', 'std'])
                          .transpose()
                          .reset_index())

    print(dataframe)

    dataframe.columns = dataframe.columns.map(str)

    dataframe_coefs = (dataframe.query('level_0.str.contains("coef")')
                                .query('level_0 != "true_val_coef"')
                                .query('not level_0.str.contains("ape")')
                                .round(3)
                                .astype(str))

    for variable in dataframe_coefs.columns:
        if variable != 'level_0' and variable != 'level_1':
            dataframe_coefs[variable] = dataframe_coefs.apply(lambda x: std_parenth_padder(x['level_1'], x[variable]), axis = 1)

    dataframe_coefs['level_0'] = dataframe_coefs.apply(lambda x: format_level_0(x.level_1, x.level_0), axis = 1)

    dataframe_coefs = (dataframe_coefs.drop(columns = 'level_1')
                                    .replace(coef_names_mapper))

    Minimal_Latex(dataframe_coefs, tables_dir + '/' + parameter + '_coefs_' + exp_mapper[exp] + '.tex')

    dataframe_apes = (dataframe.query('level_0.str.contains("ape")')
                           .query('level_1 != "std"')
                           .drop(columns = 'level_1')
                           .replace(ape_names_mapper)
                           .set_index('level_0')
                           .multiply(100)
                           .round(1)
                           .astype(str)
                           .add('%')
                           .reset_index())

    Minimal_Latex(dataframe_apes, tables_dir + '/' + parameter + '_apes_' + exp_mapper[exp] + '.tex')

In [95]:
make_param_tables(output, 'p', 'yes')

p                       level_0 level_1         5        20        50
0              mismeasured_coef    mean  1.304818  1.293894  1.300616
1              mismeasured_coef     std  0.018599  0.024976  0.034458
2       mismeasured_allvar_coef    mean  1.199485  1.042214  1.037500
3       mismeasured_allvar_coef     std  0.014301  0.015179  0.050136
4          mismeasured_avg_coef    mean  1.384074  1.311454  1.301181
5          mismeasured_avg_coef     std  0.025432  0.015345  0.048022
6                      pca_coef    mean  1.219254  1.072315  1.070803
7                      pca_coef     std  0.018575  0.001048  0.050646
8                 true_val_coef    mean  1.022568  0.972731  1.012342
9                 true_val_coef     std  0.024299  0.023804  0.045547
10                      iv_coef    mean  1.251276  1.051129  1.056732
11                      iv_coef     std  0.009356  0.012565  0.046140
12                      pca_ape    mean  0.219254  0.072315  0.070803
13                  

In [96]:
make_param_tables(output, 'p', 'no')

p                       level_0 level_1         5        20        50
0              mismeasured_coef    mean  1.283782  1.307299  1.292297
1              mismeasured_coef     std  0.020089  0.021994  0.020919
2       mismeasured_allvar_coef    mean  1.095431  1.053843  1.032973
3       mismeasured_allvar_coef     std  0.005443  0.024190  0.015546
4          mismeasured_avg_coef    mean  1.094623  1.051592  1.030931
5          mismeasured_avg_coef     std  0.005809  0.027160  0.017029
6                      pca_coef    mean  1.095827  1.052295  1.031454
7                      pca_coef     std  0.007616  0.025948  0.016926
8                 true_val_coef    mean  0.989000  1.015812  1.018516
9                 true_val_coef     std  0.016092  0.028610  0.015462
10                      iv_coef    mean  1.113225  1.057822  1.043356
11                      iv_coef     std  0.009591  0.031210  0.012600
12                      pca_ape    mean  0.095827  0.052295  0.031454
13                  

In [97]:
make_param_tables(output, 'covariance', 'yes')

covariance                      level_0 level_1      -0.9      -0.5       0.0  \
0                      mismeasured_coef    mean  0.240953  0.688289  1.010109   
1                      mismeasured_coef     std  0.013570  0.010554  0.029335   
2               mismeasured_allvar_coef    mean  0.381684  0.787294  1.007035   
3               mismeasured_allvar_coef     std  0.017160  0.016204  0.016613   
4                  mismeasured_avg_coef    mean  0.154333  0.623916  0.997804   
5                  mismeasured_avg_coef     std  0.035227  0.021724  0.031771   
6                              pca_coef    mean  0.317234  0.773750  0.996676   
7                              pca_coef     std  0.041469  0.024413  0.020028   
8                         true_val_coef    mean  0.986656  0.969266  1.001035   
9                         true_val_coef     std  0.039633  0.004051  0.007195   
10                              iv_coef    mean  0.299647  0.745506  0.995683   
11                          

In [98]:
make_param_tables(output, 'covariance', 'no')

covariance                      level_0 level_1      -0.9      -0.5       0.0  \
0                      mismeasured_coef    mean  0.222996  0.704848  1.026584   
1                      mismeasured_coef     std  0.016439  0.021707  0.000094   
2               mismeasured_allvar_coef    mean  0.502274  0.888955  1.017426   
3               mismeasured_allvar_coef     std  0.041968  0.003510  0.003012   
4                  mismeasured_avg_coef    mean  0.502976  0.889908  1.016822   
5                  mismeasured_avg_coef     std  0.039979  0.003885  0.004599   
6                              pca_coef    mean  0.502710  0.889899  1.016931   
7                              pca_coef     std  0.039979  0.004036  0.004694   
8                         true_val_coef    mean  0.988012  0.994405  1.015957   
9                         true_val_coef     std  0.047932  0.012563  0.004233   
10                              iv_coef    mean  0.456290  0.868487  1.014559   
11                          

In [99]:
make_param_tables(output, 'beta1', 'yes')

beta1                      level_0 level_1       0.1       1.0       10.0
0                 mismeasured_coef    mean  0.387305  1.304818  10.311584
1                 mismeasured_coef     std  0.016600  0.018599   0.010170
2          mismeasured_allvar_coef    mean  0.281743  1.199485  10.213327
3          mismeasured_allvar_coef     std  0.006199  0.014301   0.008460
4             mismeasured_avg_coef    mean  0.465930  1.384074  10.410618
5             mismeasured_avg_coef     std  0.012918  0.025432   0.025501
6                         pca_coef    mean  0.315518  1.219254  10.240334
7                         pca_coef     std  0.020200  0.018575   0.008588
8                    true_val_coef    mean  0.084172  1.022568  10.035408
9                    true_val_coef     std  0.003167  0.024299   0.007792
10                         iv_coef    mean  0.335861  1.251276  10.274642
11                         iv_coef     std  0.010300  0.009356   0.010326
12                         pca_ape    

In [100]:
make_param_tables(output, 'beta1', 'no')

beta1                      level_0 level_1       0.1       1.0       10.0
0                 mismeasured_coef    mean  0.380430  1.283782  10.300547
1                 mismeasured_coef     std  0.019515  0.020089   0.023988
2          mismeasured_allvar_coef    mean  0.191032  1.095431  10.132153
3          mismeasured_allvar_coef     std  0.003863  0.005443   0.047549
4             mismeasured_avg_coef    mean  0.190580  1.094623  10.132051
5             mismeasured_avg_coef     std  0.005945  0.005809   0.047415
6                         pca_coef    mean  0.191213  1.095827  10.133068
7                         pca_coef     std  0.005118  0.007616   0.047517
8                    true_val_coef    mean  0.087173  0.989000  10.029099
9                    true_val_coef     std  0.000741  0.016092   0.047099
10                         iv_coef    mean  0.209732  1.113225  10.156297
11                         iv_coef     std  0.012762  0.009591   0.050792
12                         pca_ape    

In [101]:
make_param_tables(output, 'beta2', 'yes')

beta2                      level_0 level_1       0.1       1.0      10.0
0                 mismeasured_coef    mean  1.034228  1.304818  4.042376
1                 mismeasured_coef     std  0.016310  0.018599  0.117534
2          mismeasured_allvar_coef    mean  1.023676  1.199485  3.044577
3          mismeasured_allvar_coef     std  0.007979  0.014301  0.165531
4             mismeasured_avg_coef    mean  1.038485  1.384074  4.937272
5             mismeasured_avg_coef     std  0.002995  0.025432  0.237109
6                         pca_coef    mean  1.024085  1.219254  3.261862
7                         pca_coef     std  0.009015  0.018575  0.356652
8                    true_val_coef    mean  0.997369  1.022568  0.958148
9                    true_val_coef     std  0.017835  0.024299  0.044486
10                         iv_coef    mean  1.024096  1.251276  3.559375
11                         iv_coef     std  0.005365  0.009356  0.271046
12                         pca_ape    mean  0.02408

In [102]:
make_param_tables(output, 'beta2', 'no')

beta2                      level_0 level_1       0.1       1.0      10.0
0                 mismeasured_coef    mean  1.039640  1.283782  3.841934
1                 mismeasured_coef     std  0.019253  0.020089  0.121188
2          mismeasured_allvar_coef    mean  1.027920  1.095431  2.140967
3          mismeasured_allvar_coef     std  0.009790  0.005443  0.020231
4             mismeasured_avg_coef    mean  1.026798  1.094623  2.143712
5             mismeasured_avg_coef     std  0.009160  0.005809  0.015839
6                         pca_coef    mean  1.027326  1.095827  2.146168
7                         pca_coef     std  0.009594  0.007616  0.016466
8                    true_val_coef    mean  1.020012  0.989000  1.000569
9                    true_val_coef     std  0.010127  0.016092  0.041354
10                         iv_coef    mean  1.029599  1.113225  2.344797
11                         iv_coef     std  0.011325  0.009591  0.023136
12                         pca_ape    mean  0.02732